# CounterApp

Przejdźmy do utworzenia pierwszej aplikacji i stwórzmy prosty licznik. 

<img src="https://media0.giphy.com/media/Lh9I3cXP2VdH1sFrsc/giphy.gif?cid=790b7611d0829bffb2f61d086d037fcf87f9b0f805ef3b33&rid=giphy.gif&ct=g" width="200" />

Rozpoczniemy od utworzenia interfejsu zawierającego jeden `Button` i pole `TextView`. Przechodzimy do pliku `activity_main.xml`.

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical"
    tools:context=".MainActivity">

    <TextView
        android:id="@+id/show_count"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:text="0"
        android:textSize="250sp"
        android:gravity="center"
        android:textStyle="bold"/>

    <Button
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:text="Count Up"/>
</LinearLayout>

Zauważmy kilka nowych elementów:
- `android:id` - jest identyfikatorem przez który możemy odnosić się do tego elementu w kodzie
- `gravity` - wyśrodkowanie elementu

Przejdźmy teraz do pliku `MainActivity.kt` w którym implementujemy funckjonalność głównej aktywności i dodajmy obsługę zdarzenia `onClick`. W pierwszej kolejności musimy utworzyć pole reprezentujące `TextView` - będzie to pole niemutowalne. Musimy również powiązać pole `showCount` z elementem layoutu o id `show_count`. W pierwszej kolejności robimy to wywołując metodę `findViewById` - jest to jedna z wielu dostępnych opcji. Ponieważ nasz `showCount` jest niemutowalny inicjalizację wykonujemy przez delegat `by lazy`.

In [ ]:
private val showCount: TextView by lazy{findViewById(R.id.show_count)}

Zwróćmy uwagę na wywołanie metody `findViewById`, jako argument przyjmuje ona id typu `Int`. Klasa `R` w Androidzie jest generowana automatycznie podczas kompilacji projektu i zawiera odwołania do zasobów aplikacji, takich jak pliki `xml`, pliki graficzne, stringi, style i wiele innych. Klasa `R` jest często używana do odwoływania się do zasobów aplikacji w kodzie Java lub Kotlin.

Klasa `R` jest podzielona na wiele kategorii, takich jak `R.drawable`, `R.layout`, `R.string`, `R.color` i wiele innych. Każda kategoria odpowiada innemu typowi zasobu w aplikacji. Na przykład, `R.drawable` zawiera odwołania do plików graficznych, `R.layout` zawiera odwołania do plików `xml` z opisem interfejsu użytkownika, a `R.string` zawiera odwołania do ciągów tekstowych używanych w aplikacji.

Następnie dodajmy zmienną reprezentującą stan licznika

In [ ]:
private var count = 0

Kolejnym krokiem jest wyświetlenie tekstu, w tym celu wywołujemy metodę `text`

In [ ]:
override fun onCreate(savedInstanceState: Bundle?) {
    super.onCreate(savedInstanceState)
    setContentView(R.layout.activity_main)

    showCount.text = count.toString()
}

Ostatnim krokiem jest dodanie implementacji metody `countUpButton`. W ciele klasy `MainActivity` dodajemy pole reprezentujące nasz przycisk (musimy również nadać mu id w pliku layoutu).

In [ ]:
private val increaseButton: Button by lazy { findViewById(R.id.increase_button) }

Następnie dodajemy kod obsługujący zdarzenie `onClick`

In [ ]:
override fun onCreate(savedInstanceState: Bundle?) {
    super.onCreate(savedInstanceState)
    setContentView(R.layout.activity_main)

    showCount.text = count.toString()

    increaseButton.setOnClickListener {
        count++
        showCount.text = count.toString()
    }
}

Na tym etapie możemy zbudować projekt.

<img src="https://fv9-6.failiem.lv/thumb_show.php?i=e73zs4cbf&view" width="200" />

### `Bundle`

Zwróćmy uwagę że przy zmianie orientacji urządzenia stan licznika jest resetowany

<img src="https://fv9-5.failiem.lv/thumb_show.php?i=auy6zck7f&view" width="200" />

 Jest to związane z cyklem życia aktywności - aktywność jest niszczona i odtwarzana przy każdej zmianie konfiguracji. Możemy dane zachować przy pomocy obiektu typu `Bundle`. Zwróćmy uwagę na sygnaturę metody `OnCreate(savedInstanceState: Bundle?)` - przyjmuje ona jeden argument, to właśnie w nim możemy przekazać dane których nie chcemy utracić. Jest to typ zerowalny, ponieważ przy pierwszym uruchomieniu i wywołaniu metody `OnCreate` obiekt `Bundle` zawsze będzie miał wartość `null`.

W pierwszym kroku musime zapisać dane które chcemy zachować i spakować je do obiektu `Bundle`. W tym celu mamy specjalną metodę którą możemy nadpisać w klasie `MainActivity` - metoda nazywa się `onSaveInstanceState`.

In [ ]:
override fun onSaveInstanceState(outState: Bundle) {
    super.onSaveInstanceState(outState)
    outState.putInt("counter_state", count)
}

Do obiektu typu `Bundle` (standardowo nazywanym `outState` w metodzie `onSaveInstanceState`) możemy dodać dowolną liczbę danych za pomocą odpowiednich metod, tutaj chcemy przekazać stan licznika, więc używamy metody `putInt`. Metoda ta przyjmuje dwa argumenty:
- klucz typu `String` który jest unikalnym identyfikatorem
- wartość typu odpowiadającemu wywoływnej metodzie - tutaj wykorzystując metodę `putInt` wartość będzie typu `int`

Za każdym razem przy zmianie konfiguracji urządzenia metoda `onSaveInstanceState` jest wywoływana automatycznie. Dane możemy odebrać w metodzie `OnCreate`, która przyjmuje argueemnt typu `Bundle` (nie jest to jedyna możliwość - jest również metoda `onRestoreinstanceState`). Aby rozpakować przekazane dane do zmiennej `count` przypisujemy wartość wyciągniętą z argumentu `savedInstanceState` - pamiętając o sprawdzeniu czy `Bundle` nie ma wartości `null`. Posiada ją zawsze przy pierwszym uruchomieniu. Do metody `OnCreate` dodajemy kod

In [ ]:
if (savedInstanceState != null)
    count = savedInstanceState.getInt("counter_state")

Tutaj istotna jest kolejność wywołań - wpierw próbujemy przypisać do `count` wartość z `savedInstanceState`, jeżeli to się nie uda wartość `count` pozostaje domyślna. Wartość wyjmujemy za pomocą analogicznej do `putInt` metody `getInt`, która przyjmuje jeden argument - jest to unikalny. Oczywistym jest że klucz w metodach `putInt` i `getInt` musi być ten sam. Dopiero wtedy wykonujemy aktualizację pola `showCount` (**TextView**).

Przy prawidłowej implementacji stan licznika powinien być zachowany w przypadku wystąpienia zdarzenia rotacji.

### `ViewBinding`

Ostatnim elementem naszej pierwszej aplikacji będzie wykorzyystanie `ViewBinding` do połączenia elementów laoutu z odpowiednimi polami w klasie `MainActivity`.

Metoda `findViewById` jest jednym ze sposobów uzyskania dostępu do elementów interfejsu użytkownika w Androidzie. Metoda ta służy do wyszukiwania elementu na podstawie jego identyfikatora, który jest zdefiniowany w pliku `XML` z opisem interfejsu użytkownika. Metoda ta zwraca obiekt typu `View`, który można następnie rzutować na odpowiedni typ elementu interfejsu użytkownika, np. `EditText` lub `Button`.

`ViewBinding` to narzędzie wprowadzone przez Android Studio w wersji 3.6, które umożliwia wygodniejsze i bezpieczniejsze odwoływanie się do elementów interfejsu użytkownika w kodzie aplikacji. `ViewBinding` generuje klasę `Binding` dla każdego pliku `XML` z opisem interfejsu użytkownika, która zawiera odwołania do wszystkich elementów interfejsu użytkownika zdefiniowanych w tym pliku.

W przeciwieństwie do metody `findViewById`, która jest wrażliwa na błędy podczas rzutowania obiektu `View` na konkretny typ elementu interfejsu użytkownika, `ViewBinding` zwraca już gotowe obiekty typu konkretnej klasy elementu interfejsu użytkownika. Ponadto, `ViewBinding` automatycznie generuje klasy `Binding` dla każdego pliku `XML` z opisem interfejsu użytkownika i odwołania do elementów interfejsu są odbywają się za pomocą bezpiecznych nullowalnych pól, co pozwala uniknąć problemów związanych z `NullPointerException`.

Używanie `ViewBinding` jest zwykle uważane za lepsze rozwiązanie niż użycie metody `findViewById`, ponieważ jest to bardziej bezpieczne i wygodne rozwiązanie. Jednak zastosowanie `ViewBinding` wymaga dodatkowej konfiguracji projektu i wygenerowania odpowiednich klas `Binding` dla plików `XML` z opisem interfejsu użytkownika.

Rozpoczniemy od dodania odpowiedniego wpisu do pliku `build.gradle(Module)`

In [ ]:
android {
    ...
    buildFeatures {
        viewBinding = true
    }
}

Przejdźmy do klasy `MainActivity`, w pierwszym kroku musimy utworzyć instancję automatycznie utworzonej klasy - plik do którego się odnosimy nosi nazwę `activity_main.xml`, więc klasa będzie nosić nazwę `ActivityMainBinding`

In [ ]:
private val binding by lazy { ActivityMainBinding.inflate(layoutInflater) }

`ActivityMainBinding` to klasa, która została wygenerowana automatycznie przez system `ViewBinding` na podstawie pliku `XML` z opisem interfejsu użytkownika (layoutu). Metoda `inflate` z klasy `ActivityMainBinding` tworzy obiekt typu `ActivityMainBinding`, który zawiera odwołania do wszystkich elementów interfejsu użytkownika zdefiniowanych w pliku `XML`.

`layoutInflater` jest wykorzystywany do utworzenia obiektu `ActivityMainBinding`. `layoutInflater` jest obiektem, który umożliwia tworzenie obiektów z plików `XML` z opisem interfejsu użytkownika.

Następnie możemy pozbyć się dwóch pól

In [ ]:
//private val showCount: TextView by lazy{findViewById(R.id.show_count)}
//private val increaseButton: Button by lazy { findViewById(R.id.increase_button) }

W kolejnym kroku musimy zmodyfikować argument metody `setContentVierw`

In [ ]:
//setContentView(R.layout.activity_main)
setContentView(binding.root)

Metoda `setContentView(R.layout.activity_main)` ustawia widok za pomocą pliku `XML` z opisem interfejsu użytkownika, który jest przechowywany w folderze `res/layout`. W tym przypadku, system Android odczytuje plik `activity_main.xml` z folderu `res/layout` i wstawia go jako widok do głównego okna aplikacji.

Natomiast metoda `setContentView(binding.root)` ustawia widok, używając obiektu klasy `ActivityMainBinding`, który został utworzony przy użyciu `ViewBinding`. Ta metoda odwołuje się do właściwości `root` obiektu `ActivityMainBinding`, która reprezentuje najwyższy poziom widoku (najczęściej jest to `ConstraintLayout`, lub najbardziej *"zewnętrzny"* element layoutu), który został utworzony przez `ViewBinding`.

Teraz przez `binding` możemy odnosić się do wszystkich pól zdefiniowanych (i posiadających `id`) w pliku `activity_main.xml`

In [ ]:
override fun onCreate(savedInstanceState: Bundle?) {
    super.onCreate(savedInstanceState)
    //setContentView(R.layout.activity_main)
    setContentView(binding.root)

    if (savedInstanceState != null)
        count = savedInstanceState.getInt("counter_state")

    //showCount.text = count.toString()
    binding.showCount.text = count.toString()

    //increaseButton.setOnClickListener {
    binding.increaseButton.setOnClickListener { 
        count++
        //showCount.text = count.toString()
        binding.showCount.text = count.toString()
    }
}

Mopżna również spotkać się z wykorrzystaniem funkcji `apply`

In [ ]:
binding.apply {
    showCount.text = count.toString()
    increaseButton.setOnClickListener {
        count++
        binding.showCount.text = count.toString()
    }
}